# 목표

- 추천시스템의 개념과 목적을 이해한다.
- Implicit 라이브러리를 활용하여 Matrix Factorization(이하 MF) 기반의 추천 모델을 만들어 본다.
- 음악 감상 기록을 활용하여 비슷한 아티스트를 찾고 아티스트를 추천해 본다.
- 추천 시스템에서 자주 사용되는 데이터 구조인 CSR Matrix을 익힌다
- 유저의 행위 데이터 중 Explicit data와 Implicit data의 차이점을 익힌다.
- 새로운 데이터셋으로 직접 추천 모델을 만들어 본다.

In [2]:
# tsv파일을 사용. tab으로 구분되어 있는 형식.
import pandas as pd
import os

fname = os.getenv('HOME') + '/aiffel/recommendata_iu/data/lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv'
col_names = ['user_id', 'artist_MBID', 'artist', 'play']   # 임의로 지정한 컬럼명
data = pd.read_csv(fname, sep='\t', names= col_names)      # sep='\t'로 주어야 tsv를 열 수 있습니다.  
data.head(10)

,user_id,artist_MBID,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,3bd73256-3905-4f3a-97e2-8b341527f805,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,f2fb0ff0-5679-42ec-a55c-15109ce6e320,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,b3ae82c2-e60b-4551-a76d-6620f1b456aa,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,3d6bbeb7-f90e-4d10-b440-e153c0d10b53,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,bbd2ffd7-17f4-4506-8572-c1ea58c3f9a8,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,8bfac288-ccc5-448d-9573-c33ea2aa5c30,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,6531c8b1-76ea-4141-b270-eb1ac5b41375,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,21f3573f-10cf-44b3-aeaa-26cccd8448b5,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,c5db90c4-580d-4f33-b364-fbaa5a3a58b5,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,0639533a-0402-40ba-b6e0-18b067198b73,lunachicks,403


In [3]:
# 사용하는 컬럼만 남기고 제거

using_cols = ['user_id', 'artist', 'play']
data = data[using_cols]
data.head(10)

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


In [4]:
# 검색을 쉽게하기 위해 아티스트 문자열을 소문자로 바꿔 줍시다.
data['artist'] = data['artist'].str.lower() # 검색을 쉽게하기 위해 아티스트 문자열을 소문자로 바꿔줍시다.
data.head(10)

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


In [5]:
# 첫 번째 유저가 어떤 아티스트의 노래를 듣는지 확인해 봅시다.
condition = (data['user_id']== data.loc[0, 'user_id'])
data.loc[condition]

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


추천 모델을 만들기 전에 데이터의 기본적인 정보를 보고 갑시다. 아래 항목들을 확인해 보고 싶습니다.

- 유저수, 아티스트수, 인기 많은 아티스트
- 유저들이 몇 명의 아티스트를 듣고 있는지에 대한 통계
- 유저 play 횟수 중앙값에 대한 통계

In [6]:
# 유저 수
data['user_id'].nunique()
print(data['user_id'].nunique())
# 아티스트 수
data['artist'].nunique()
print(data['artist'].nunique())
# 인기 많은 아티스트
artist_count = data.groupby('artist')['user_id'].count()
artist_count.sort_values(ascending=False).head(30)

358868
291346


artist
radiohead                77254
the beatles              76245
coldplay                 66658
red hot chili peppers    48924
muse                     46954
metallica                45233
pink floyd               44443
the killers              41229
linkin park              39773
nirvana                  39479
system of a down         37267
queen                    34174
u2                       33206
daft punk                33001
the cure                 32624
led zeppelin             32295
placebo                  32072
depeche mode             31916
david bowie              31862
bob dylan                31799
death cab for cutie      31482
arctic monkeys           30348
foo fighters             30144
air                      29795
the rolling stones       29754
nine inch nails          28946
sigur rós                28901
green day                28732
massive attack           28691
moby                     28232
Name: user_id, dtype: int64

In [7]:
# 유저별 play횟수 중앙값에 대한 통계
# 유저별 몇 명의 아티스트를 듣고 있는지에 대한 통계
user_count = data.groupby('user_id')['artist'].count()
user_count.describe()

count    358868.000000
mean         48.863234
std           8.524272
min           1.000000
25%          46.000000
50%          49.000000
75%          51.000000
max         166.000000
Name: artist, dtype: float64

In [8]:
# 유저별 play횟수 중앙값에 대한 통계
user_median = data.groupby('user_id')['play'].median()
user_median.describe()

count    358868.000000
mean        142.187676
std         213.089902
min           1.000000
25%          32.000000
50%          83.000000
75%         180.000000
max       50142.000000
Name: play, dtype: float64

우리는 본인의 음악 취향과 가장 유사한 아티스트를 추천받고 싶습니다. 유튜브 뮤직 등 추천시스템들은 이를 위해서 처음 가입하는 사용자의 취향과 유사한 아티스트 정보를 5개 이상 입력받는 과정을 거치게 하는 경우가 많습니다.
우리도 동일한 과정이 필요합니다. 우리가 만든 추천시스템의 추후 검증 과정을 위해, '나는 A를 좋아한다.'라는 정보를 위 데이터셋에 5개 이상 추가해 주어야 합니다. 저는 아래와 같이 제가 좋아하는 유명한 아티스트들을 기존의 데이터에 추가해서 넣겠습니다.

In [9]:
# 본인이 좋아하시는 아티스트 데이터로 바꿔서 추가하셔도 됩니다! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
my_favorite = ['black eyed peas' , 'maroon5' ,'jason mraz' ,'coldplay' ,'beyoncé']

# 'zimin'이라는 user_id가 위 아티스트의 노래를 30회씩 들었다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['zimin']*5, 'artist': my_favorite, 'play':[30]*5})

if not data.isin({'user_id':['zimin']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    data = data.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

data.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,artist,play
17535650,"sep 20, 2008",turbostaat,12
17535651,"sep 20, 2008",cuba missouri,11
17535652,"sep 20, 2008",little man tate,11
17535653,"sep 20, 2008",sigur rós,10
17535654,"sep 20, 2008",the smiths,10
0,zimin,black eyed peas,30
1,zimin,maroon5,30
2,zimin,jason mraz,30
3,zimin,coldplay,30
4,zimin,beyoncé,30


사람이 태어나면 주민등록번호가, 학교에 가면 출석번호가 있듯이 데이터의 관리를 쉽게 하기 위해 번호를 붙여주고 싶습니다. 우리가 다루는 데이터에서는 user와 artist 각각에 번호를 붙이고 싶습니다. 보통 이런 작업을 indexing이라고 합니다.

In [10]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = data['user_id'].unique()
artist_unique = data['artist'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
artist_to_idx = {v:k for k,v in enumerate(artist_unique)}

In [11]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['zimin'])    # 358869명의 유저 중 마지막으로 추가된 유저이니 358868이 나와야 합니다. 
print(artist_to_idx['black eyed peas'])

358868
376


In [12]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_artist_data = data['artist'].map(artist_to_idx.get).dropna()
if len(temp_artist_data) == len(data):
    print('artist column indexing OK!!')
    data['artist'] = temp_artist_data
else:
    print('artist column indexing Fail!!')

data

user_id column indexing OK!!
artist column indexing OK!!


,user_id,artist,play
0,0,0,2137
1,0,1,1099
2,0,2,897
3,0,3,717
4,0,4,706
...,...,...,...
0,358868,376,30
1,358868,270115,30
2,358868,3746,30
3,358868,62,30


우리의 추천시스템은 사용자들이 아이템을 얼마나 선호하는지를 모델링하기를 원합니다. 그러자면 사용자의 아이템 선호도를 말해 주는 유저 행동 데이터셋이 필요합니다. 만약 우리 데이터가 좋아요나 별점처럼 선호도를 명시적(explicit)으로 나타내는 것이라면 참 좋겠지만, 지금 우리가 가지고 있는 데이터는 사용자가 아티스트의 곡을 몇 번 플레이했나 하는 것뿐입니다.

In [13]:
# 1회만 play한 데이터의 비율을 보는 코드
only_one = data[data['play']<2]
one, all_data = len(only_one), len(data)
print(f'{one},{all_data}')
print(f'Ratio of only_one over all data is {one/all_data:.2%}')  # f-format에 대한 설명은 https://bit.ly/2DTLqYU

147740,17535660
Ratio of only_one over all data is 0.84%


이에 우리 모델은 암묵적 데이터의 해석을 2가지 규칙에 적용하여 볼 것 입니다.
1. 한 번이라도 들었으면 선호라고 판단.
2. 많이 재생한 아티스트에 대해 가중치를 주어서 더 확실히 좋아한다고 판단.

# matrix factorization
기본 아이디어는 아래 그림과 같습니다. (m,n) 사이즈의 행렬 R을 (m,k) 사이즈의 행렬 P와 (k,n) 사이즈의 행렬 Q로 분해한다면 R이란 그저 P와 Q의 행렬곱으로 표현 가능할 수 있다는 간단한 아이디어입니다. 대체로 k는 m이나 n보다 훨씬 작은 값이기 때문에 계산량 측면으로도 훨씬 유리해집니다. 하지만 아이디어의 단순함에도 불구하고 MF 모델은 성능이 준수하고 Scalability가 좋아서 많이 사용되는 모델입니다.

# compressed sparse row matrix
유저 수 X 아티스트 수만큼의 정보 안에는 유저가 들어보지 않은 아티스트에 대한 정보까지 모두 행렬에 포함되어 계산되기 때문입니다. 아무리 많은 아티스트를 들어 본 유저도 수백 명을 넘어갈 만큼의 아티스트를 들어보지 못했는데, 총 아티스트는 29만 명이 넘기 때문에 평가행렬 내의 대부분의 공간은 0으로 채워집니다. 이런 행렬을 Sparse Matrix라고 합니다. 이런 메모리 낭비를 최소화하기 위해서는 유저가 들어본 아티스트에 대해서만 정보만을 저장하면서 전체 행렬 형태를 유추할 수 있는 데이터 구조가 필요합니다.

CSR Matrix는 Sparse한 matrix에서 0이 아닌 유효한 데이터로 채워지는 데이터의 값과 좌표 정보만으로 구성하여 메모리 사용량을 최소화하면서도 Sparse한 matrix와 동일한 행렬을 표현할 수 있도록 하는 데이터 구조입니다. CSR에 대한 자세한 설명은 다음 링크를 참고해 주세요. 혹시 설명이 부족하면 그 아래 StackOverflow의 설명이 보완적인 역할을 해줄 것입니다.

In [14]:
# csr_matrix
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_artist = data['artist'].nunique()

csr_data = csr_matrix((data.play, (data.user_id, data.artist)), shape= (num_user, num_artist))
csr_data

<358869x291347 sparse matrix of type '<class 'numpy.longlong'>'
	with 17535578 stored elements in Compressed Sparse Row format>

# MF 모델 학습하기
이전 스텝에서 설명한 Matrix Factorization 모델을 implicit 패키지를 사용해서 학습해 봅니다

- `implicit` 패키지는 이전 스텝에서 설명한 암묵적 dataset을 사용하는 다양한 모델을 굉장히 빠르게 학습할 수 있는 패키지 입니다.
- 이 패키지에 구현된 `als(AlternatingLeastSquares)`모델을 사용하겠습니다. Matrix Factorization에서 쪼개진 두 Feature Matrix를 한꺼번에 훈련하는 것은 잘 수렴하지 않기 때문에, 한쪽을 고정시키고 다른 쪽을 하습하는 방식을 번갈아 수행하는 AlternatingLeastSquares 방식이 효과적.

In [15]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

AlternatingLeastSquares 클래스의 `__init__` 파라미터를 살펴보겠습니다. 
1. factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지 
2. regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지 
3. use_gpu : GPU를 사용할 것인지 
4. iterations : epochs와 같은 의미입니다. 데이터를 몇 번 반복해서 학습할 것인지

1,4를 늘릴수록 학습데이터를 잘 학습하게 되지만 과적합의 우려가 있으니 좋은 값을 찾아야 합니다.

In [16]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [17]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<291347x358869 sparse matrix of type '<class 'numpy.longlong'>'
	with 17535578 stored elements in Compressed Sparse Column format>

In [18]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

모델 학습이 끝났습니다! 이제 2가지를 아래 사항을 살펴보도록 하겠습니다.

1. Zimin 벡터와 black eyed peas의 벡터를 어떻게 만들고 있는지
2. 두 벡터를 곱하면 어떤 값이 나오는지

In [20]:
zimin, black_eyed_peas = user_to_idx['zimin'], artist_to_idx['black eyed peas']
zimin_vector, black_eyed_peas_vector = als_model.user_factors[zimin], als_model.item_factors[black_eyed_peas]

print('슝=3')

슝=3


In [21]:
zimin_vector

array([-0.02023125, -0.24335198, -0.20056592,  0.5148617 , -0.9615922 ,
        0.37285578,  0.17426929,  0.4483591 , -0.49836877, -0.9070103 ,
       -0.47247916,  1.0657609 , -0.11880158, -0.15865687,  0.6512244 ,
       -0.17525038, -1.1349663 ,  0.69290906,  0.4783158 ,  1.1454022 ,
        0.28376144, -0.03632002, -0.5352076 ,  0.4157492 ,  0.16453482,
       -0.32628265, -0.1577616 ,  0.5200111 , -0.8967516 ,  1.0531526 ,
       -1.4718935 ,  0.03750828, -0.60793436,  0.8021652 ,  0.2625607 ,
       -0.16177104, -1.1607246 ,  0.14270942, -0.37688267,  0.5056662 ,
        0.7433653 , -0.3765016 ,  0.6741875 ,  0.39488393, -0.473661  ,
        0.37807915, -0.39296183, -0.00530928, -0.4593094 , -0.14102511,
        0.10487065,  0.4394087 , -0.11004175, -0.28050786, -0.09875654,
        0.9384219 , -0.6062194 , -0.3655289 ,  0.03267035, -0.07229819,
       -0.6243476 ,  0.5978654 , -0.45088273, -0.10023332,  0.28634912,
        1.5673666 , -1.7925959 ,  0.05830567,  1.5760572 , -1.23

In [22]:
black_eyed_peas_vector

array([-1.09021305e-04,  1.51555864e-02, -7.40725268e-03,  1.74692157e-03,
        1.13820545e-02,  1.23748695e-02,  9.76101216e-03, -3.60381627e-03,
       -5.92352808e-06, -1.43850809e-02,  7.24007981e-03,  2.56234985e-02,
        7.27052288e-03,  1.33683961e-02,  1.93422679e-02, -9.60190082e-04,
        7.80596817e-03,  1.25328815e-02, -2.87966570e-03,  2.44887508e-02,
        2.47604903e-02,  1.10788364e-02, -7.02720042e-03,  1.16945123e-02,
        4.38776193e-03,  9.80524905e-03,  3.00255651e-03,  7.79777067e-03,
       -1.19735515e-02,  1.64777599e-02, -1.13962721e-02,  1.32838711e-02,
        4.22809040e-03,  1.57255493e-02,  4.33221972e-03,  8.40300694e-03,
        1.30383205e-03,  2.12479923e-02,  2.16234894e-03,  1.47446673e-02,
        2.14543100e-02,  5.34952432e-03,  1.55104799e-02,  3.50044016e-03,
        1.04263343e-03,  4.25845757e-03, -1.14786057e-02,  1.49460081e-02,
       -7.82158226e-04,  1.76671054e-02,  1.02073690e-02,  2.65685003e-02,
        4.56131389e-03,  

In [23]:
# zimin과 black_eyed_peas를 내적하는 코드
np.dot(zimin_vector, black_eyed_peas_vector)

0.52287936

이런, 1이 나와야 될 것 같은데 한참 낮은 수치인 0.49정도가 나왔습니다. factors를 늘리거나 iterations를 늘려야 할 것 같습니다. 이후 진행되는 내용은 일단 이렇게 학습된 모델을 사용하겠습니다.

만약 이 두 수치를 늘려 1에 가깝게 나왔다면 이 모델이 잘 학습되었다고 볼 수 있을까요?

정답은 아직 검증되지 않았다입니다. 학습 데이터에 대해서 fitting은 되었지만 아직 보지 못한 데이터에 대한, 예를 들어 제가 queen을 선호할지에 대한 예측을 잘할지 못할지는 검증하지 않았기 때문입니다.

In [25]:
#그렇다면 모델이 저의 queen에 대한 선호도를 어떻게 예측할지 한 번 보겠습니다

queen = artist_to_idx['queen']
queen_vector = als_model.item_factors[queen]
np.dot(zimin_vector, queen_vector)

0.31026235

# 비슷한 아티스트 찾기
`AlternatingLeastSquares` 클래스에 구현되어 있는 `similar_items` 메서드를 통하여 비슷한 아티스트를 찾습니다. 처음으로는 제가 좋아하는 `coldplay`로 찾아보겠습니다.

In [26]:
favorite_artist = 'coldplay'
artist_id = artist_to_idx[favorite_artist]
similar_artist = als_model.similar_items(artist_id, N=15)
similar_artist

[(62, 1.0000001),
 (277, 0.98414767),
 (28, 0.9810952),
 (5, 0.9725954),
 (217, 0.9706849),
 (473, 0.96646124),
 (418, 0.9642381),
 (490, 0.9638969),
 (247, 0.96033144),
 (694, 0.94974995),
 (55, 0.9462606),
 (910, 0.9457035),
 (1018, 0.9415705),
 (782, 0.94093174),
 (531, 0.93711287)]

In [27]:
# (아티스트의 id, 유사도) Tuple 로 반환하고 있습니다. 아티스트의 id를 다시 아티스트의 이름으로 매핑 시켜 주겠습니다.

#artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
idx_to_artist = {v:k for k,v in artist_to_idx.items()}
[idx_to_artist[i[0]] for i in similar_artist]

['coldplay',
 'muse',
 'the killers',
 'red hot chili peppers',
 'radiohead',
 'placebo',
 'u2',
 'oasis',
 'the beatles',
 'foo fighters',
 'arctic monkeys',
 'nirvana',
 'the smashing pumpkins',
 'the white stripes',
 'depeche mode']

In [28]:
def get_similar_artist(artist_name: str):
    artist_id = artist_to_idx[artist_name]
    similar_artist = als_model.similar_items(artist_id)
    similar_artist = [idx_to_artist[i[0]] for i in similar_artist]
    return similar_artist

print("슝=3")

슝=3


In [29]:
get_similar_artist('2pac')

['2pac',
 'the game',
 'notorious b.i.g.',
 '50 cent',
 'nas',
 'jay-z',
 'snoop dogg',
 'dr. dre',
 'ludacris',
 'bone thugs-n-harmony']

# 유저에게 아티스트 추천하기

`AlternatingLeastSquares` 클래스에 구현되어 있는 `recommend` 메서드를 통하여 제가 좋아할 만한 아티스트를 추천받습니다. `filter_already_liked_items` 는 유저가 이미 평가한 아이템은 제외하는 Argument입니다.

In [30]:
user = user_to_idx['zimin']
# recommend에서는 user*item CSR Matrix를 받습니다.
artist_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
artist_recommended

[(350, 0.44232008),
 (369, 0.44087318),
 (627, 0.4345893),
 (550, 0.42838347),
 (274, 0.41539025),
 (354, 0.41476554),
 (355, 0.41367763),
 (409, 0.40292823),
 (621, 0.39546248),
 (2249, 0.3951149),
 (1800, 0.3905121),
 (564, 0.38955742),
 (391, 0.3770113),
 (24, 0.37093377),
 (901, 0.3682701),
 (5555, 0.366819),
 (382, 0.3597081),
 (434, 0.35724908),
 (637, 0.35046685),
 (724, 0.34818864)]

In [31]:
[idx_to_artist[i[0]] for i in artist_recommended]

['rihanna',
 'justin timberlake',
 'maroon 5',
 'britney spears',
 'michael jackson',
 'nelly furtado',
 'madonna',
 'amy winehouse',
 'alicia keys',
 'katy perry',
 'lady gaga',
 'kanye west',
 'christina aguilera',
 'jack johnson',
 'pink',
 'timbaland',
 'mika',
 'eminem',
 'akon',
 'lily allen']

rihanna를 추천해주고 있네요. 모델은 왜 rihanna를 추천해줬을까요? AlternatingLeastSquares 클래스에 구현된 explain 메소드를 사용하면 제가 기록을 남긴 데이터 중 이 추천에 기여한 정도를 확인할 수 있습니다.

In [32]:
rihanna = artist_to_idx['rihanna']
explain = als_model.explain(user, csr_data, itemid=rihanna)

이 method는 추천한 콘텐츠의 점수에 기여한 다른 콘텐츠와 기여도(합이 콘텐츠의 점수가 됩니다.)를 반환합니다. 어떤 아티스트들이 이 추천에 얼마나 기여하고 있는 걸까요?

In [33]:
[(idx_to_artist[i[0]], i[1]) for i in explain[1]]

[('beyoncé', 0.2309511001561303),
 ('black eyed peas', 0.12413168603174139),
 ('jason mraz', 0.05040678584275883),
 ('coldplay', 0.04014449939712659),
 ('maroon5', 8.568792610741295e-05)]

beyoncé와 black eyed peas가 가장 크게 기여했네요. 둘 다 흑인 아티스트이고 beyoncé의 경우는 같은 성별이라는 점도 크게 작용하지 않았을까 싶습니다.

눈치채신 분도 있겠지만 결과가 조금 이상합니다. filter_already_liked_items=True로 했는데 앞에 maroon 5는 제가 이미 평가한 아이템이 나오고 있습니다. 이것은 implicit 버전 0.4.2에서 생긴 버그입니다. 코드는 사람이 만들기 때문에 이렇게 실수할 가능성이 있습니다.
오픈 소스 생태계에서는 누구든 이런 문제점을 지적하고 Pull Request를 통하여 수정을 제안할 수 있습니다.
많이들 사용하는 Tensorflow나 numpy, pandas도 여러 작은 버그들이 있습니다. 이런 유명한 패키지의 오류를 고쳐서 Pull Request를 하여 Contribute 한다면 좋은 이력이 되기도 합니다.

# project

In [35]:
# 데이터 준비와 전처리

import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [36]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [38]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [39]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [40]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [41]:
# 분석

# 유니크한 영화 개수
print(ratings['movie_id'].nunique())
# 유니크한 사용자 수
print(ratings['user_id'].nunique())
# 인기영화 30개
movie_count = ratings.groupby('movie_id')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

3628
6039


movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: user_id, dtype: int64

In [42]:
# 내가 선호하는 영화 5가지를 골라서 rating에 추가

my_favorite = [593, 858, 1, 2396, 1580]

my_movielist = pd.DataFrame({'user_id': ['song']*5, 'movie_id': my_favorite, 'count':[5]*5})

if not ratings.isin({'user_id':['song']})['user_id'].any():  
    ratings = ratings.append(my_movielist)                           

user_unique = ratings['user_id'].unique()
movie_unique = ratings['movie_id'].unique()

user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}


temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data    
else:
    print('user_id column indexing Fail!!')

temp_movie_data = ratings['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('movie column indexing OK!!')
    ratings['movie_id'] = temp_movie_data
else:
    print('artist column indexing Fail!!')

ratings

user_id column indexing OK!!
movie column indexing OK!!


,user_id,movie_id,count,timestamp
0,0,0,5,978300760.0
1,0,1,3,978302109.0
2,0,2,3,978301968.0
3,0,3,4,978300275.0
4,0,4,5,978824291.0
...,...,...,...,...
0,6039,121,5,NaN
1,6039,607,5,NaN
2,6039,40,5,NaN
3,6039,126,5,NaN


In [43]:
# csr matrix를 만들기

from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.movie_id)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

In [44]:
# als_model = alternating least squares 모델을 직접 구성하여 훈련

from implicit.als import AlternatingLeastSquares
import os
import numpy as np

os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

csr_data_transpose = csr_data.T

als_model.fit(csr_data_transpose)


  0%|          | 0/15 [00:00<?, ?it/s]

In [45]:
# 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

song, magica = user_to_idx['song'], movie_to_idx[858]
song_vector, movie_vector = als_model.user_factors[song], als_model.item_factors[magica]

np.dot(song_vector, movie_vector)


0.65465784

In [47]:
# 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

movie_id = movie_to_idx[1]
similar_movie = als_model.similar_items(movie_id, N=15)

idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]


def get_similar_movie(movie_name: int):
    movie_id = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

get_similar_movie(1)

[1, 3114, 34, 2355, 588, 1265, 364, 595, 2321, 1923]

In [48]:
# 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

user = user_to_idx['song']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[idx_to_movie[i[0]] for i in movie_recommended]

glass_bottom_boat = movie_to_idx[3144]
explain = als_model.explain(user, csr_data, itemid=glass_bottom_boat)

[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[(858, 0.01617663243580724),
 (2396, 0.006247086288981636),
 (593, 0.0036164501050051662),
 (1580, -0.0016364131696693057),
 (1, -0.00835220099383057)]

# 회고
https://github.com/iphfly1030/first-repository/blob/4260c65de20f08f13af2966cdd67b8ae2abc5535/Exploration%208%20%EC%A0%9C%EC%B6%9C%EC%9A%A9.ipynb 을 참고해서 프로젝트를 진행했습니다.

cv나 nlp는 이제 익숙해져 가는 단계라 이제 어느정도 코드를 따라 이해하며 읽어나갈수 있는데, 추천시스템은 처음보는 개념에다 지금까지 해왔던 방식이 아닌 다른 방식으로 문제 해결을 해가서, cv나 nlp처럼 추천 시스템의 여러 코드들을 경험해가면서 익술해 져야 겠다는 생각이 듭니다. 특히 mf이나 als부분은 이해가 쉽게 가지 않아서 애를 좀 먹었는데, 구글링 하니까 여전히 이해는 안되지만 뭔가 100중에 1정도 이해하게된 느낌이 들었습니다. 더 열심히 해야할거 같습니다..